In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# Seed value
seed_value = 55

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [3]:
dataset_file_name = 'human_dataset.txt'
y_train_full = np.load('../Saved Data/Training Set/labels.npy')
X_train_full = np.load('../Saved Data/Training Set/dna_spectral_representation.npy')

In [4]:
k = 5
input_vector_size = np.power(4, k)

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size = 0.1,
                                                        random_state=20)

# Model : LeNet5 like Architecture CNN for Text Classification

In [6]:
model = keras.Sequential()

model.add(keras.layers.Conv1D(filters=30, kernel_size=(5), activation='relu', input_shape=(input_vector_size, 1),padding="same"))
model.add(keras.layers.MaxPooling1D())

model.add(keras.layers.Conv1D(filters=15, kernel_size=(5), activation='relu', input_shape=(int(input_vector_size / 2), 1), padding="same"))
model.add(keras.layers.MaxPooling1D())

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(units=256, activation='relu'))

model.add(keras.layers.Dense(units=128, activation='relu'))

model.add(keras.layers.Dense(units=64, activation='relu'))

model.add(keras.layers.Dense(units=int(7), activation = 'softmax'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1024, 30)          180       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 512, 30)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 512, 15)           2265      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 256, 15)          0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 3840)              0         
                                                                 
 dense (Dense)               (None, 256)               9

In [8]:
opt = keras.optimizers.SGD(learning_rate=0.045, momentum=0.0)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [9]:
# earlyStop = keras.callbacks.EarlyStopping(monitor='loss', patience=2, mode="min")

history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid), batch_size=32)

Epoch 1/20
99/99 [==============================] - 3s 26ms/step - loss: 1.7239 - accuracy: 0.3663 - val_loss: 1.5113 - val_accuracy: 0.4644
Epoch 2/20
99/99 [==============================] - 3s 26ms/step - loss: 1.3296 - accuracy: 0.5369 - val_loss: 1.3252 - val_accuracy: 0.4815
Epoch 3/20
99/99 [==============================] - 3s 26ms/step - loss: 0.9855 - accuracy: 0.6743 - val_loss: 0.8403 - val_accuracy: 0.6781
Epoch 4/20
99/99 [==============================] - 2s 25ms/step - loss: 0.6555 - accuracy: 0.7805 - val_loss: 0.6037 - val_accuracy: 0.7920
Epoch 5/20
99/99 [==============================] - 3s 25ms/step - loss: 0.4224 - accuracy: 0.8671 - val_loss: 0.5760 - val_accuracy: 0.8091
Epoch 6/20
99/99 [==============================] - 2s 25ms/step - loss: 0.4565 - accuracy: 0.8693 - val_loss: 0.5046 - val_accuracy: 0.8462
Epoch 7/20
99/99 [==============================] - 3s 28ms/step - loss: 0.2461 - accuracy: 0.9305 - val_loss: 0.4815 - val_accuracy: 0.8661
Epoch 8/20
99

In [10]:
# data = history.history
# pd.DataFrame(data).plot(figsize=(8, 6))
# plt.grid(True)
# plt.gca().set_ylim(0, 2)
# plt.show()

In [11]:
model.save("../Saved Data/Model/")
model.save("../Saved Data/Model Architecture/model.h5")

INFO:tensorflow:Assets written to: ../Saved Data/Model/assets


INFO:tensorflow:Assets written to: ../Saved Data/Model/assets
